In [2]:
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import backend as K
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.utils import np_utils

import tensorflow as tf
from tensorflow import *

import numpy as np
import pickle

from sklearn.metrics import log_loss
from scale_layer import Scale
import sys

sys.setrecursionlimit(3000)
K.set_image_dim_ordering('tf')
K.set_image_data_format('channels_last')

Using TensorFlow backend.


In [3]:
file = open("train_y_res.dat",'rb')
train_y_raw = pickle.load(file)
file.close()

file = open("test_y_res.dat",'rb')
test_y_raw = pickle.load(file)
file.close()

file = open("train_x_res.dat",'rb')
train_x_raw2 = pickle.load(file)
file.close()

file = open("test_x_res.dat",'rb')
test_x_raw2 = pickle.load(file)
file.close()

In [ ]:
train_x_raw = np.reshape(train_x_raw2, (train_x_raw2.shape[0], train_x_raw2.shape[1], train_x_raw2.shape[2], 1))
test_x_raw = np.reshape(test_x_raw2, (test_x_raw2.shape[0], test_x_raw2.shape[1], test_x_raw2.shape[2], 1))

fake_col_train = np.zeros((train_x_raw.shape[0], train_x_raw.shape[1], train_x_raw.shape[2], 2))
fake_col_test = np.zeros((train_x_raw.shape[0], train_x_raw.shape[1], train_x_raw.shape[2], 2))

train_x = np.concatenate((train_x_raw, fake_col_train), axis=3)
test_x = np.concatenate((test_x_raw, fake_col_test), axis=3)

In [ ]:
# Convert class vectors to binary class matrices
train_y = np_utils.to_categorical(train_y_raw, 2)
test_y = np_utils.to_categorical(test_y_raw, 2)

In [ ]:
batch_size = 20
nb_classes = output_dims = 2
nb_epoch = 3

In [ ]:
# this could also be the output a different Keras model or layer
input_tensor = Input(shape=(267, 267, 3))

# create the base pre-trained model
base_model = InceptionResNetV2(input_tensor=input_tensor,weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

# train the model on the new data for a few epochs
model.fit(train_x, train_y, epochs=nb_epoch, batch_size=batch_size, 
          validation_data=(test_x, test_y))

In [ ]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(train_x, train_y, epochs=nb_epoch, batch_size=batch_size, 
          validation_data=(test_x, test_y))

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model_resnet.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_resnet.h5")
print("Saved model to disk")